### 새로나온 책(1-10page)
- 제목
- 저자
- 역자
- 출간일
- 페이지
- 정가
- 판가
- 마일리지

In [169]:
import requests
from urllib.parse import quote
import pandas as pd
from bs4 import BeautifulSoup

In [190]:
base_url = 'https://www.hanbit.co.kr'         
sub_url = '/store/books/new_book_list.html'
url = base_url + sub_url
res = requests.get(url)

In [171]:
soup = BeautifulSoup(res.text, 'html.parser')

In [172]:
lis = soup.select('.sub_book_list')
len(lis)

20

In [189]:
li = lis[0]
href = li.find('a')['href']
href

'/store/books/look.php?p_code=B6694310162'

In [191]:
res = requests.get(base_url + href)
# sub_html = sub_res.text
book_soup = BeautifulSoup(res.text, 'html.parser')

In [193]:
info = book_soup.select_one('.store_product_info_box')
title = li.select_one('.book_tit').find('a').get_text()
title

'하루 한 장 초등과학 365'

In [194]:
book_lis = info.select('.info_list > li')
author = book_lis[0].find('span').get_text()
author

'치바 가즈요시 '

In [195]:
if len(book_lis) == 5:
    translator = '-'
else:
    translator = book_lis[1].find('span').get_text().strip()
translator

'허영은 , 유우종 (감수)'

In [196]:
date = book_lis[-4].find('span').get_text().strip()
date

'2021-12-14'

In [197]:
page = book_lis[-3].find('span').get_text().strip().split()[0]
page = int(page)
page

416

In [198]:
payments = book_soup.select_one('.payment_box.curr').find_all('p')
price = payments[0].select_one('.pbr').get_text().strip()
price

'19,800원'

In [199]:
sales = payments[1].select_one('.pbr').get_text().split('(')[0]
sales

'17,820원'

In [185]:
mileage = payments[2].select_one('.pbr').get_text().split()[0]
mileage

'990점'

In [200]:
from tqdm.notebook import tqdm

lines = []
for page in tqdm(range(1,11)):
    url = f'{base_url}{sub_url}?page={page}'
    res = requests.get(url)
    soup = BeautifulSoup(res.text, 'html.parser')
    lis = soup.select('.sub_book_list')
    
    for li in lis:
        book_url = li.find('a')['href']
        res = requests.get(base_url + book_url)
        book_soup = BeautifulSoup(res.text, 'html.parser')

        info = book_soup.select_one('.store_product_info_box')
        title = info.find('h3').string
        book_lis = info.select('.info_list > li')
        author = book_lis[0].find('span').get_text().strip()
        if len(book_lis) == 5:
            translator = '-'
        else:
            translator = book_lis[1].find('span').get_text().strip()
        date = book_lis[-4].find('span').get_text().strip()
        page = book_lis[-3].find('span').get_text().strip().split()[0]
        
        payments = book_soup.select_one('.payment_box.curr').find_all('p')
        price = payments[0].select_one('.pbr').get_text().strip()
        sales = payments[1].select_one('.pbr').get_text().split('(')[0]
        mileage = payments[2].select_one('.pbr').get_text().split()[0]

        lines.append([title,author,translator,date,int(page),
                      convert_unit(price),convert_unit(sales),convert_unit(mileage)])

  0%|          | 0/10 [00:00<?, ?it/s]

In [186]:
def convert_unit(s):
    return int(s.replace(',','').replace('원','').replace('점',''))

In [201]:
df = pd.DataFrame(lines, columns=['책제목','저자','번역','출간일','쪽수','정가','판매가','마일리지'])
df.head()

,책제목,저자,번역,출간일,쪽수,정가,판매가,마일리지
0,하루 한 장 초등과학 365,치바 가즈요시,"허영은 , 유우종 (감수)",2021-12-14,416,"19,800","17,820",990
1,"다시, 배우다",폴 김,-,2021-12-10,304,"16,000","14,400",800
2,당당한 디자인 결정을 위한 9가지 방법,톰 그리버,"김민성 , 곽서희",2021-12-10,376,"22,000","19,800","1,100"
3,365 부모 말하기 연습 일력(스프링북),"박재연 , 공인영",-,2021-12-01,384,"18,000","16,200",900
4,공학도를 위한 수치해석(8판),"Steven C.Chapra , Raymond P. Canale","나양 , 이승배 , 임윤철",2021-11-30,600,"36,000","36,000","1,080"


- Bug fix

In [202]:
len(lis)

20

In [203]:
lis[15]

<li class="sub_book_list">
<div class="view_box">
<!--
            <span class="view_box_block" onclick="location.href='/store/books/look.php?p_code=B9718869266';">
            -->
<span class="view_box_block">
<span class="img_hidden_a">
<span class="pop_quick_menu">
<a href="/store/books/look.php?p_code=B9718869266"><img alt="상세보기" src="https://www.hanbit.co.kr/images/common/icon_view_detail.png"/></a>
</span>
<span class="price">28,000<span>원</span></span>
<span class="pop_quick_bg" onclick="location='/store/books/look.php?p_code=B9718869266'"></span>
</span>
<!-- 예약도서인 경우 -->
<!-- ebook이 도서인 경우 -->
<img alt="" class="thumb" src="/data/books/B9718869266_m.jpg"/>
</span>
<div class="info">
<p class="book_tit"><a href="/store/books/look.php?p_code=B9718869266">IT CookBook, C 언어 일취월장(2판)</a></p>
<p class="book_writer">신윤환 </p>
</div>
</div>
</li>

In [204]:
href = lis[15].find('a')['href']
res = requests.get(base_url + href)
book_soup = BeautifulSoup(res.text, 'html.parser')

In [205]:
book_lis = info.select('.info_list > li')
book_dict = {}
for li in book_lis:
    items = li.get_text().split(':')
    book_dict[items[0].strip()] = items[1].strip()
    
book_dict

{'저자': '앤드루 호프먼',
 '번역': '최용',
 '출간': '2021-02-19',
 '페이지': '372 쪽',
 'ISBN': '9791162243930',
 '물류코드': '10393'}

In [206]:
lines = []
for page in tqdm(range(1,11)):
    url = f'{base_url}{sub_url}?page={page}'
    res = requests.get(url)
    soup = BeautifulSoup(res.text, 'html.parser')
    lis = soup.select('.sub_book_list')
    
    for li in lis:
        book_url = li.find('a')['href']
        res = requests.get(base_url + book_url)
        book_soup = BeautifulSoup(res.text, 'html.parser')

        info = book_soup.select_one('.store_product_info_box')
        title = info.find('h3').string
        book_lis = info.select('.info_list > li')
        book_dict = {}
        for li in book_lis:
            items = li.get_text().split(':')
            book_dict[items[0].strip()] = items[1].strip()
        author = book_dict['저자']
        translator = book_dict['번역'] if '번역' in book_dict.keys() else '-' 
        date = book_dict['출간']
        page = book_dict['페이지'].split()[0]
        
        payments = book_soup.select_one('.payment_box.curr').find_all('p')
        price = payments[0].select_one('.pbr').get_text().strip()
        sales = payments[1].select_one('.pbr').get_text().split('(')[0]
        mileage = payments[2].select_one('.pbr').get_text().split()[0]

        lines.append([title,author,translator,date,int(page),
                      convert_unit(price),convert_unit(sales),convert_unit(mileage)])

  0%|          | 0/10 [00:00<?, ?it/s]

In [207]:
df = pd.DataFrame(lines, columns=['제목','저자','번역','출간일','페이지','정가','판매가','마일리지'])
df.tail()

,제목,저자,번역,출간일,페이지,정가,판매가,마일리지
195,"IT CookBook, C 언어 일취월장(2판)",신윤환,-,2021-02-24,800,"28,000","28,000",840
196,소비자행동,김학윤,-,2021-02-23,660,"35,000","35,000","1,050"
197,재미있고 빠른 어휘력 동화 – 두근두근 이 마음은 뭘까,"김세실 , 김도윤(그림)",-,2021-02-20,104,"12,000","10,800",600
198,재미있고 빠른 어휘력 동화 – 키득키득 재밌는 의성어 의태어,"김예실 , 김희선(그림)",-,2021-02-20,104,"12,000","10,800",600
199,웹 애플리케이션 보안,앤드루 호프먼,최용,2021-02-19,372,"30,000","27,000","1,500"


In [208]:
df.to_csv('한빛도서.csv', index=False)